# Experiment 04-01

In [110]:
import sys
from glob import glob
import numpy as np
import pyopencl
import hashlib

## Helper Functions

In [111]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [112]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [113]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

## Setup the device and context

In [114]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


In [115]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)
gpu_queue = pyopencl.CommandQueue(gpu_context)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x600003df1600 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Load the Program

In [116]:
build_options = [
    '-I', '.',    
]

program_files = [
    'inc_hash_sha256.cl',
    'inc_common.cl',
    'inc_platform.cl',
    'solver.cl'
]

gpu_program = build_program(program_files, gpu_context, build_options)

# show the kernel names
program_kernel_names = gpu_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: sha256_test;increment_int;hello;number_to_string


## Run Program

### Test Int Conversion

In [117]:
# set the input values
x = np.uint8(116)
y = np.zeros(shape=(1,), dtype=np.uint32)

# create the input and output tuples
input_tuples = (
    (x, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, x.nbytes)),
)

output_tuples = (
    (y, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.WRITE_ONLY, y.itemsize)),
)

# build the program
gpu_program = build_program(program_files, gpu_context, build_options)

# set the global and local sizes
global_size = (1,)
local_size = None

# run the program
run_ocl_kernel(gpu_queue, gpu_program.increment_int, global_size, input_tuples,
    output_tuples, local_size)

print(type(x), x)
print(type(y[0]), y[0])

<class 'numpy.uint8'> 116
<class 'numpy.uint32'> 118


### Test Number to Char Array Conversion

In [118]:
number = np.uint64(123456)
print(np.iinfo(number.dtype).max, number.itemsize)

18446744073709551615 8


In [157]:
number = np.uint64(321234562)
output = np.zeros(20, dtype=np.uint8)

# create the input and output tuples
input_tuples = (
    (number, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, number.nbytes)),
)

output_tuples = (
    (output, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.WRITE_ONLY, output.nbytes)),
)

# build the program
gpu_program = build_program(program_files, gpu_context, build_options)

# set the global and local sizes
global_size = (1,)
local_size = None

# run the program
run_ocl_kernel(gpu_queue, gpu_program.itoa, global_size, input_tuples,
    output_tuples, local_size)

print(type(number), number)
print(type(output), output)
print(f'"%s"' % ''.join([chr(item) for item in output]))

<class 'numpy.uint64'> 321234562
<class 'numpy.ndarray'> [51 50 49 50 51 52 53 54 50  0  0  0  0  0  0  0  0  0  0  0]
"321234562           "


## SHA256 Testing

In [ ]:
plaintext = 'this is a description of the latest block'

# compute the hash for comparison
print(f'Plaintext: "{plaintext}"')
print(hashlib.sha256(plaintext.encode('utf-8')).hexdigest())

# get the bytes to encode and compute the hash for testing
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_size = np.int32(len(plaintext_bytes))
print(plaintext_bytes)

Plaintext: "this is a description of the latest block"
2a10a8810de7777ed32fdc9be92caf54dd8cf7f62311c95b874fab57302ae79c
[116 104 105 115  32 105 115  32  97  32 100 101 115  99 114 105 112 116
 105 111 110  32 111 102  32 116 104 101  32 108  97 116 101 115 116  32
  98 108 111  99 107]


In [ ]:
# # create the input tuples
# input_tuples = (
#     (plaintext_bytes, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, plaintext_bytes.nbytes)),
#     (plaintext_size, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, sys.getsizeof(plaintext_size))),
# )

In [ ]:
# # create the input tuples
# input_tuples = (
#     (plaintext_bytes, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, plaintext_bytes.nbytes)),
#     (plaintext_size, pyopencl.Buffer(gpu_context, pyopencl.mem_flags.READ_ONLY, sys.getsizeof(plaintext_size))),
# )

# output_tuples = (
# )

# # execute the program
# global_size = (1,)
# local_size = None

# gpu_queue = pyopencl.CommandQueue(gpu_context)

# #run_ocl_kernel(gpu_queue, gpu_program.hello, global_size, (), (), local_size)
# run_ocl_kernel(gpu_queue, gpu_program.sha256_test, global_size, input_tuples, output_tuples, local_size)